## 5 - Train Mode / Eval Mode

In [1]:
import torch, torchvision
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F


There some widely used methods in deep learning that shows different behaviour in training than testing phase. 
For example, dropout shows stochastic behaviour (removes neurons randomly with drop probability `p`) in the training phase but it multiplies all
the weights with `p` in the testing phase to scale the weights. (Actually, there is another implementation of dropout (and `torch.Dropout` uses that implementation. It has similar behaviour with the one explained above and you are recommended to do a little research about it.) 

For now, what you need to know is;
- Some operations have different behavior in the test phase and the training phase
- Dropout is one of them, it shows a stochastic behaviour only in the training phase, but not in test phase.
- Networks are created in training phase by default.
- There are 2 modes of a network. `train` mode for training and `eval` mode for testing or validation.


In [2]:
class NetworkWithDropout(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = torch.nn.Linear(20, 5)
        self.dropout = torch.nn.Dropout(p = 0.2)
        self.layer2 = torch.nn.Linear(5, 2)
    
    def forward(self, x):
        x = self.layer1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.layer2(x)
        return x
    
network = NetworkWithDropout()
print(network)

NetworkWithDropout(
  (layer1): Linear(in_features=20, out_features=5, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (layer2): Linear(in_features=5, out_features=2, bias=True)
)


In [3]:
dummy_input = torch.randn(4, 20)
dummy_output = network(dummy_input)
print(f'Input Shape: {dummy_input.shape}')
print(f'Output Shape: {dummy_output.shape}')

Input Shape: torch.Size([4, 20])
Output Shape: torch.Size([4, 2])


To see how dropout has a stochastic behavior by default, run the cell below multiple times.
Note: 
- Ctrl + Enter runs cell
- Shift + Enter runs cell and proceeds to the next cell

In [4]:
dummy_output = network(dummy_input)
print(dummy_output)

tensor([[-0.0845, -0.0893],
        [-0.1345,  0.2107],
        [-0.0572, -0.0692],
        [ 0.0949, -0.2109]], grad_fn=<AddmmBackward>)


To set (all the layers of) network to "eval" mode, call `.eval()` method from the network.

In [5]:
network.eval()

NetworkWithDropout(
  (layer1): Linear(in_features=20, out_features=5, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (layer2): Linear(in_features=5, out_features=2, bias=True)
)

Now that network is in the "eval" mode, run the cell below multiple times to see how dropout layer stopped its stochastic behavior.

In [6]:
dummy_output = network(dummy_input)
print(dummy_output)

tensor([[-0.3755, -0.2792],
        [-0.0933,  0.1769],
        [-0.0348, -0.0560],
        [ 0.0869, -0.1693]], grad_fn=<AddmmBackward>)


To set (all the layers of) network to "train" mode again, call `.train()` method from the network.

In [7]:
network.train()

NetworkWithDropout(
  (layer1): Linear(in_features=20, out_features=5, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (layer2): Linear(in_features=5, out_features=2, bias=True)
)

Now that network is in the default "train" mode, run the cell below multiple times to see how dropout layer acts as default.

In [8]:
dummy_output = network(dummy_input)
print(dummy_output)

tensor([[-0.3436, -0.2621],
        [ 0.0593,  0.0081],
        [-0.0572, -0.0692],
        [ 0.3041, -0.0928]], grad_fn=<AddmmBackward>)


---

A remarkable amount of bugs arise from forgetting to call `.eval()` before test/validation or forgetting to call `.train()` after test/validation.

Also, check this tweet from Andrej Karpathy to see the most common mistakes when training a neural network.

https://twitter.com/karpathy/status/1013244313327681536